# [Create Agent With OpenAPI](https://learn.microsoft.com/en-us/python/api/overview/azure/ai-projects-readme?view=azure-python-preview#create-agent-with-openapi)
AzureFunctionTool contains the input and output queues of azure function and the description of input parameters.

Inspired by [sample_agents_openapi.py](https://github.com/Azure/azure-sdk-for-python/blob/azure-ai-projects_1.0.0b4/sdk/ai/azure-ai-projects/samples/agents/sample_agents_openapi.py)

In [1]:
import os, jsonref
from dotenv import load_dotenv # requires python-dotenv
# import logging
# logging.basicConfig(level=logging.INFO) # Configure logging 

load_dotenv("./../config/credentials_my.env")
model_name =  "gpt-4o-0513" # https://learn.microsoft.com/en-us/azure/ai-services/agents/how-to/tools/azure-ai-search?tabs=azurecli%2Cpython&pivots=overview-azure-ai-search
index_name =  "ms-surface-specs"

print(f'Project Connection String: <...{os.environ["PROJECT_CONNECTION_STRING"][-30:]}>')

Project Connection String: <...mai04-rg;mmai-hub04-prj01-fvye>


In [2]:
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import OpenApiTool, OpenApiAnonymousAuthDetails # <<<<<<<<<<<<<<< SPECIFIC FOR OPENAPI
from azure.identity import DefaultAzureCredential

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=os.environ["PROJECT_CONNECTION_STRING"]
)

project_client.scope

{'subscription_id': 'eca2eddb-0f0c-4351-a634-52751499eeea',
 'resource_group_name': 'mmai04-rg',
 'project_name': 'mmai-hub04-prj01-fvye'}

In [3]:
# https://github.com/Azure/azure-sdk-for-python/blob/azure-ai-projects_1.0.0b4/sdk/ai/azure-ai-projects/samples/agents/weather_openapi.json
with open("./openapi/weather_openapi.json", "r") as f:
    openapi_spec = jsonref.loads(f.read())

openapi_spec

{'openapi': '3.1.0',
 'info': {'title': 'get weather data',
  'description': 'Retrieves current weather data for a location based on wttr.in.',
  'version': 'v1.0.0'},
 'servers': [{'url': 'https://wttr.in'}],
 'auth': [],
 'paths': {'/{location}': {'get': {'description': 'Get weather information for a specific location',
    'operationId': 'GetCurrentWeather',
    'parameters': [{'name': 'location',
      'in': 'path',
      'description': 'City or location to retrieve the weather for',
      'required': True,
      'schema': {'type': 'string'}},
     {'name': 'format',
      'in': 'query',
      'description': 'Always use j1 value for this parameter',
      'required': True,
      'schema': {'type': 'string', 'default': 'j1'}}],
    'responses': {'200': {'description': 'Successful response',
      'content': {'text/plain': {'schema': {'type': 'string'}}}},
     '404': {'description': 'Location not found'}},
    'deprecated': False}}},
 'components': {'schemes': {}}}

In [4]:
# Create Auth object for the OpenApiTool (note that connection or managed identity auth setup requires additional setup in Azure)
auth = OpenApiAnonymousAuthDetails()
auth

{'type': 'anonymous'}

In [5]:
# Initialize agent OpenApi tool using the read in OpenAPI spec
openapi = OpenApiTool(
    name="get_weather", spec=openapi_spec, description="Retrieve weather information for a location", auth=auth
)
print(f"openapi.definitions: {openapi.definitions}")
print(f"openapi.resources: {openapi.resources}")

openapi.definitions: [{'type': 'openapi', 'openapi': {'name': 'get_weather', 'description': 'Retrieve weather information for a location', 'spec': {'openapi': '3.1.0', 'info': {'title': 'get weather data', 'description': 'Retrieves current weather data for a location based on wttr.in.', 'version': 'v1.0.0'}, 'servers': [{'url': 'https://wttr.in'}], 'auth': [], 'paths': {'/{location}': {'get': {'description': 'Get weather information for a specific location', 'operationId': 'GetCurrentWeather', 'parameters': [{'name': 'location', 'in': 'path', 'description': 'City or location to retrieve the weather for', 'required': True, 'schema': {'type': 'string'}}, {'name': 'format', 'in': 'query', 'description': 'Always use j1 value for this parameter', 'required': True, 'schema': {'type': 'string', 'default': 'j1'}}], 'responses': {'200': {'description': 'Successful response', 'content': {'text/plain': {'schema': {'type': 'string'}}}}, '404': {'description': 'Location not found'}}, 'deprecated': 

In [6]:
# Check if we already have any agents defined
project_client.agents.list_agents()

{'object': 'list', 'data': [], 'first_id': None, 'last_id': None, 'has_more': False}

In [7]:
# Create agent with AI search tool and process assistant run
agent = project_client.agents.create_agent(
    model=model_name,
    name="my-assistant",
    instructions="You are a helpful assistant",
    tools=openapi.definitions
)

agent.items

<bound method _MyMutableMapping.items of {'id': 'asst_IuTkQtjfJ6DzZDM3NYXnaM9H', 'object': 'assistant', 'created_at': 1735837075, 'name': 'my-assistant', 'description': None, 'model': 'gpt-4o-0513', 'instructions': 'You are a helpful assistant', 'tools': [{'type': 'openapi', 'openapi': {'name': 'get_weather', 'description': 'Retrieve weather information for a location', 'spec': {'openapi': '3.1.0', 'info': {'title': 'get weather data', 'description': 'Retrieves current weather data for a location based on wttr.in.', 'version': 'v1.0.0'}, 'servers': [{'url': 'https://wttr.in'}], 'auth': [], 'paths': {'/{location}': {'get': {'description': 'Get weather information for a specific location', 'operationId': 'GetCurrentWeather', 'parameters': [{'name': 'location', 'in': 'path', 'description': 'City or location to retrieve the weather for', 'required': True, 'schema': {'type': 'string'}}, {'name': 'format', 'in': 'query', 'description': 'Always use j1 value for this parameter', 'required': Tr

In [8]:
# Check again if the agent created above is now present here
project_client.agents.list_agents()

{'object': 'list', 'data': [{'id': 'asst_IuTkQtjfJ6DzZDM3NYXnaM9H', 'object': 'assistant', 'created_at': 1735837075, 'name': 'my-assistant', 'description': None, 'model': 'gpt-4o-0513', 'instructions': 'You are a helpful assistant', 'tools': [{'type': 'openapi', 'openapi': {'name': 'get_weather', 'description': 'Retrieve weather information for a location', 'spec': {'openapi': '3.1.0', 'info': {'title': 'get weather data', 'description': 'Retrieves current weather data for a location based on wttr.in.', 'version': 'v1.0.0'}, 'servers': [{'url': 'https://wttr.in'}], 'auth': [], 'paths': {'/{location}': {'get': {'description': 'Get weather information for a specific location', 'operationId': 'GetCurrentWeather', 'parameters': [{'name': 'location', 'in': 'path', 'description': 'City or location to retrieve the weather for', 'required': True, 'schema': {'type': 'string'}}, {'name': 'format', 'in': 'query', 'description': 'Always use j1 value for this parameter', 'required': True, 'schema':

In [9]:
# Create thread for communication
thread = project_client.agents.create_thread()
print(f"Created thread: {thread}")

Created thread: {'id': 'thread_akFnNEZuM0nSLqCXLVeTz0jo', 'object': 'thread', 'created_at': 1735837076, 'metadata': {}, 'tool_resources': {}}


In [10]:
# Create message to thread
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content="Quale sarà la temperatura massima a Chiavari il 20 giugno? Se il giorno non è ancora disponibile, indica esattamente quale è l'ultima data per la quale sei in grado di rispondere.",
)
print(f"Created message: {message}")

Created message: {'id': 'msg_VHnO2RHHQAQ0DhdlRfA8r4J1', 'object': 'thread.message', 'created_at': 1735837077, 'assistant_id': None, 'thread_id': 'thread_akFnNEZuM0nSLqCXLVeTz0jo', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': "Quale sarà la temperatura massima a Chiavari il 20 giugno? Se il giorno non è ancora disponibile, indica esattamente quale è l'ultima data per la quale sei in grado di rispondere.", 'annotations': []}}], 'attachments': [], 'metadata': {}}


In [11]:
%%time
# Create and process agent run in thread with tools
run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
print(f"Run finished with status: {run.status}. Run: {run}")

if run.status == "failed":
    # Check if you got "Rate limit is exceeded.", then you want to get more quota
    print(f"Run failed: {run.last_error}")

Run finished with status: RunStatus.COMPLETED. Run: {'id': 'run_FpO9aqap3sdVXoIskkwmeakx', 'object': 'thread.run', 'created_at': 1735837077, 'assistant_id': 'asst_IuTkQtjfJ6DzZDM3NYXnaM9H', 'thread_id': 'thread_akFnNEZuM0nSLqCXLVeTz0jo', 'status': 'completed', 'started_at': 1735837078, 'expires_at': None, 'cancelled_at': None, 'failed_at': None, 'completed_at': 1735837080, 'required_action': None, 'last_error': None, 'model': 'gpt-4o-0513', 'instructions': 'You are a helpful assistant', 'tools': [{'type': 'openapi', 'openapi': {'name': 'get_weather', 'description': 'Retrieve weather information for a location', 'spec': {'openapi': '3.1.0', 'info': {'title': 'get weather data', 'description': 'Retrieves current weather data for a location based on wttr.in.', 'version': 'v1.0.0'}, 'servers': [{'url': 'https://wttr.in'}], 'auth': [], 'paths': {'/{location}': {'get': {'description': 'Get weather information for a specific location', 'operationId': 'GetCurrentWeather', 'parameters': [{'name

In [12]:
# Fetch and log all messages
messages = project_client.agents.list_messages(thread_id=thread.id)
for m in messages.text_messages:
    print(m.text)

{'value': 'Non posso prevedere la temperatura massima del 20 giugno. Tuttavia, posso fornirti le previsioni meteo fino al 4 gennaio 2025. La temperatura massima prevista per Chiavari il 4 gennaio 2025 è di 9°C.', 'annotations': []}
{'value': "Quale sarà la temperatura massima a Chiavari il 20 giugno? Se il giorno non è ancora disponibile, indica esattamente quale è l'ultima data per la quale sei in grado di rispondere.", 'annotations': []}


In [13]:
# Get the last message from the sender
last_msg = messages.get_last_text_message_by_sender("assistant")
if last_msg:
    print(f"Last Message: {last_msg.text.value}")

Last Message: Non posso prevedere la temperatura massima del 20 giugno. Tuttavia, posso fornirti le previsioni meteo fino al 4 gennaio 2025. La temperatura massima prevista per Chiavari il 4 gennaio 2025 è di 9°C.


In [14]:
run_steps = project_client.agents.list_run_steps(run_id=run.id, thread_id=thread.id)
run_steps_data = run_steps['data']
for rs in run_steps["data"]:
    print(rs, '\n')

{'id': 'step_vHzd5cuUuTpdBpR68GZikb8E', 'object': 'thread.run.step', 'created_at': 1735837079, 'run_id': 'run_FpO9aqap3sdVXoIskkwmeakx', 'assistant_id': 'asst_IuTkQtjfJ6DzZDM3NYXnaM9H', 'thread_id': 'thread_akFnNEZuM0nSLqCXLVeTz0jo', 'type': 'message_creation', 'status': 'completed', 'cancelled_at': None, 'completed_at': 1735837080, 'expires_at': None, 'failed_at': None, 'last_error': None, 'step_details': {'type': 'message_creation', 'message_creation': {'message_id': 'msg_PRSqLbmM9e4l2QtrT1NGmWY9'}}, 'usage': {'prompt_tokens': 9246, 'completion_tokens': 59, 'total_tokens': 9305}} 

{'id': 'step_jj9OqBJthllgISwj8kG8juOS', 'object': 'thread.run.step', 'created_at': 1735837078, 'run_id': 'run_FpO9aqap3sdVXoIskkwmeakx', 'assistant_id': 'asst_IuTkQtjfJ6DzZDM3NYXnaM9H', 'thread_id': 'thread_akFnNEZuM0nSLqCXLVeTz0jo', 'type': 'tool_calls', 'status': 'completed', 'cancelled_at': None, 'completed_at': 1735837079, 'expires_at': None, 'failed_at': None, 'last_error': None, 'step_details': {'typ

# START teardown

In [15]:
project_client.agents.list_agents()

{'object': 'list', 'data': [{'id': 'asst_IuTkQtjfJ6DzZDM3NYXnaM9H', 'object': 'assistant', 'created_at': 1735837075, 'name': 'my-assistant', 'description': None, 'model': 'gpt-4o-0513', 'instructions': 'You are a helpful assistant', 'tools': [{'type': 'openapi', 'openapi': {'name': 'get_weather', 'description': 'Retrieve weather information for a location', 'spec': {'openapi': '3.1.0', 'info': {'title': 'get weather data', 'description': 'Retrieves current weather data for a location based on wttr.in.', 'version': 'v1.0.0'}, 'servers': [{'url': 'https://wttr.in'}], 'auth': [], 'paths': {'/{location}': {'get': {'description': 'Get weather information for a specific location', 'operationId': 'GetCurrentWeather', 'parameters': [{'name': 'location', 'in': 'path', 'description': 'City or location to retrieve the weather for', 'required': True, 'schema': {'type': 'string'}}, {'name': 'format', 'in': 'query', 'description': 'Always use j1 value for this parameter', 'required': True, 'schema':

In [16]:
print(f"deleting trhead: {thread}...")
project_client.agents.delete_thread(thread.id)

deleting trhead: {'id': 'thread_akFnNEZuM0nSLqCXLVeTz0jo', 'object': 'thread', 'created_at': 1735837076, 'metadata': {}, 'tool_resources': {}}...


{'id': 'thread_akFnNEZuM0nSLqCXLVeTz0jo', 'object': 'thread.deleted', 'deleted': True}

In [17]:
# Delete all agents
for pca in project_client.agents.list_agents()['data']:
    print(f"Deleting agent {pca.name} ({pca.id})...")
    project_client.agents.delete_agent(pca.id)

Deleting agent my-assistant (asst_IuTkQtjfJ6DzZDM3NYXnaM9H)...


# HIC SUNT LEONES